In [1]:
import src
import torch
import onnx

In [2]:
ENCODER = 'resnet152'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multicalss segmentation
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu') 

In [3]:
model = src.FPN(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS,
    in_channels=1,
    classes=1,
    activation=ACTIVATION,
)

In [4]:
checkpoint = torch.load('./checkpoint/best_model_without_kfold.pth')
model.load_state_dict(checkpoint['model_state_dict'])
checkpoint['best_IoU']

0.44076838547533204

In [9]:
x = torch.randn(1, 1, 768, 768, requires_grad=True)

In [10]:
model.eval()
torch.onnx.export(
    model, x, "FPN_trained.onnx", 
    export_params=True, 
    verbose=True,
    input_names = [ "actual_input_1" ] + [ "learned_%d" % i for i in range(20)],
    output_names = [ "output1" ])

c:\Users\Dmitry\Desktop\code\analysis_trus_images\env_trus\lib\site-packages\torch\nn\functional.py:2515: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  _verify_batch_size([input.size(0) * input.size(1) // num_groups, num_groups] + list(input.size()[2:]))


In [11]:
# Load the ONNX model
model = onnx.load("FPN_trained.onnx")

# Check that the model is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph torch_jit (
  %actual_input_1[FLOAT, 1x1x768x768]
) initializers (
  %decoder.p5.weight[FLOAT, 256x2048x1x1]
  %decoder.p5.bias[FLOAT, 256]
  %decoder.p4.skip_conv.weight[FLOAT, 256x1024x1x1]
  %decoder.p4.skip_conv.bias[FLOAT, 256]
  %decoder.p3.skip_conv.weight[FLOAT, 256x512x1x1]
  %decoder.p3.skip_conv.bias[FLOAT, 256]
  %decoder.p2.skip_conv.weight[FLOAT, 256x256x1x1]
  %decoder.p2.skip_conv.bias[FLOAT, 256]
  %decoder.seg_blocks.0.block.0.block.0.weight[FLOAT, 128x256x3x3]
  %decoder.seg_blocks.0.block.1.block.0.weight[FLOAT, 128x128x3x3]
  %decoder.seg_blocks.0.block.2.block.0.weight[FLOAT, 128x128x3x3]
  %decoder.seg_blocks.1.block.0.block.0.weight[FLOAT, 128x256x3x3]
  %decoder.seg_blocks.1.block.1.block.0.weight[FLOAT, 128x128x3x3]
  %decoder.seg_blocks.2.block.0.block.0.weight[FLOAT, 128x256x3x3]
  %decoder.seg_blocks.3.block.0.block.0.weight[FLOAT, 128x256x3x3]
  %segmentation_head.0.weight[FLOAT, 1x128x1x1]
  %segmentation_head.0.bias[FLOAT, 1]
  %onnx::Conv_1642[FLO